In [2]:
import pandas as pd
import psycopg2
import traceback


class Conexion_BD:
    def __init__(self, host,port,name,user,password):
        self.host = host
        self.port = port
        self.name = name
        self.user = user
        self.password = password
        self.estado = "Correcto"
        self.conexion = ''
        self.Estado_Conexion = self.crear_conexion()

    def crear_conexion(self):
        cxn = True
        try:
            self.conexion = psycopg2.connect(
                host=self.host,
                port=self.port,
                database=self.name,
                user=self.user,
                password=self.password)
        except Exception as e:
            self.estado = f"Error de conexion: \nServer: {self.host}\nDATABASE: {self.name}"
            cxn = False
        return cxn

    def consultar_bd(self, consulta):
        df = None
        self.estado = "Correcto"
        try:
            cursor = self.conexion.cursor()
            cursor.execute(consulta)
            records = cursor.fetchall()
            columnas = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(records, columns=columnas)
            cursor.close()
        except Exception as e:
            self.estado = f"Error en consulta: {traceback.format_exc()}"
        return df

    def cerrar_conexion(self):
        self.conexion.close()        
        self.Estado_Conexion = False


In [3]:
db_host = 'database-iot-pry20241050.clguogmy6dz7.us-east-2.rds.amazonaws.com'
db_port = 5432
db_name = 'iotdatabase'
db_user = 'pry20241050'
db_password = 'tH6gn6c59OX9'

In [4]:
bd = Conexion_BD(db_host,db_port,db_name,db_user,db_password)

In [5]:
bd.Estado_Conexion

True

In [6]:
consulta = """SELECT * 
FROM medicion 
WHERE date BETWEEN '2024-08-25' AND '2024-09-30' """

In [7]:
consulta_sensores = """SELECT A.id,B.name FROM sensor A LEFT JOIN tipo_gas B ON A.gas_type_id = B.id"""

In [8]:
df_datos = bd.consultar_bd(consulta)

In [9]:
df_sensores = bd.consultar_bd(consulta_sensores)

In [10]:
df_datos = df_datos[["id","date","sensor_id","value"]]

In [11]:
df_merge = pd.merge(df_datos,df_sensores,how='left',left_on='sensor_id',right_on='id')

In [12]:
df_merge = df_merge[["id_x","date","name","value"]]

In [13]:
df_merge["fecha"] = pd.to_datetime(df_merge["date"])

In [14]:
df_merge['fecha'] = df_merge['fecha'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [15]:
df_merge["fecha"] = pd.to_datetime(df_merge["date"])
df_merge['fecha'] = df_merge['fecha'].dt.tz_convert('Etc/GMT+5')

In [16]:
df_merge = df_merge[["id_x","fecha","name","value"]]

In [17]:
df_merge

,id_x,fecha,name,value
0,8749,2024-08-26 19:08:52-05:00,MONOXIDO DE CARBONO,0.3280
1,8750,2024-08-26 19:08:52-05:00,DIOXIDO DE NITROGENO,16.9960
2,8751,2024-08-26 19:08:52-05:00,DIOXIDO DE AZUFRE,16.9670
3,8752,2024-08-26 19:08:52-05:00,MATERIAL PARTICULADO 2.5,34.8920
4,8753,2024-08-26 19:08:52-05:00,MATERIAL PARTICULADO 10,35.4160
...,...,...,...,...
8797,17546,2024-08-31 21:20:45-05:00,DIOXIDO DE NITROGENO,1.5780
8798,17547,2024-08-31 21:20:45-05:00,DIOXIDO DE AZUFRE,17.1160
8799,17548,2024-08-31 21:20:45-05:00,MATERIAL PARTICULADO 2.5,100.7310
8800,17549,2024-08-31 21:20:45-05:00,MATERIAL PARTICULADO 10,106.9820


In [18]:
df_merge.fecha.dt.day.unique()

array([26, 29, 30, 31])

In [112]:
df_29_agosto = df_merge.query("fecha.dt.day==29").copy()
df_30_agosto = df_merge.query("fecha.dt.day==30").copy()
df_31_agosto = df_merge.query("fecha.dt.day==31").copy()

In [113]:
df_29_agosto.shape

(2922, 4)

In [114]:
df_30_agosto.shape

(1422, 4)

In [115]:
df_31_agosto.shape

(4404, 4)

In [40]:
df_31_agosto.fecha.dt.hour.unique()

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

In [80]:
ruta = r"C:\Users\ROGGER\Desktop\Datos Senamhi\Datos31.xlsx"

In [81]:
datos31 = pd.read_excel(ruta)

In [83]:
datos31 = datos31.drop(columns={"fecha"})

In [84]:
datos31["hora"] = datos31["hora"].astype(str)

In [85]:
datos31['hour'] = datos31['hora'].str.split(':').str[0].astype(int)

In [116]:
df_31_agosto['fecha_hora'] = df_31_agosto['fecha'].dt.floor('H')

C:\Users\ROGGER\AppData\Local\Temp\ipykernel_9796\2069443698.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_31_agosto['fecha_hora'] = df_31_agosto['fecha'].dt.floor('H')


In [117]:
df_promedio = df_31_agosto.groupby(['name', 'fecha_hora'])['value'].mean().reset_index()

In [118]:
df_promedio.loc[df_promedio["name"]=='TEMPERATURA',"name"] = "T"

In [119]:
df_promedio.loc[df_promedio["name"]=='DIOXIDO DE AZUFRE',"name"] = "SO2"

In [120]:
df_promedio.loc[df_promedio["name"]=='DIOXIDO DE NITROGENO',"name"] = "NO2"

In [121]:
df_promedio.loc[df_promedio["name"]=='MONOXIDO DE CARBONO',"name"] = "CO"

In [122]:
df_promedio.loc[df_promedio["name"]=='MATERIAL PARTICULADO 10',"name"] = "PM10"

In [123]:
df_promedio.loc[df_promedio["name"]=='MATERIAL PARTICULADO 2.5',"name"] = "PM2.5"

In [124]:
masa_no2 = 46.01
masa_so2 = 64.06
masa_co = 28.01

In [125]:
df_promedio['value'] = df_promedio['value'].astype(float)

In [126]:
df_promedio.loc[df_promedio["name"] == 'NO2', "value"] = 0.0409 * masa_no2 * df_promedio.loc[df_promedio["name"] == 'NO2', "value"]

In [127]:
df_promedio.loc[df_promedio["name"] == 'SO2', "value"] = 0.0409 * masa_so2 * df_promedio.loc[df_promedio["name"] == 'SO2', "value"]

In [128]:
df_promedio.loc[df_promedio["name"] == 'CO', "value"] = 0.0409 * masa_co * df_promedio.loc[df_promedio["name"] == 'CO', "value"]

In [129]:
copia = df_promedio.copy()

In [130]:
copia['fecha_hora'] = pd.to_datetime(copia['fecha_hora'], format='%d/%m/%Y %H:%M')
pivot_df = copia.pivot(index='fecha_hora', columns='name', values='value')
pivot_df.reset_index(inplace=True)

In [131]:
pivot_df['fecha'] = pivot_df['fecha_hora'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [132]:
pivot_df['hora'] = pivot_df['fecha_hora'].dt.strftime('%H:00')

In [133]:
pivot_df = pivot_df[["fecha","hora","PM2.5","PM10","CO","NO2","SO2"]]

In [134]:
pivot_df[["hora","PM2.5","PM10","CO","NO2","SO2"]]

name,hora,PM2.5,PM10,CO,NO2,SO2
0,07:00,94.848150,110.330450,0.000000,6.473047,53.355828
1,08:00,85.546528,97.517698,1.040386,35.222671,49.160716
2,09:00,66.480642,71.134189,1.896588,49.837261,50.277353
3,10:00,56.346660,58.750849,0.375608,33.769560,49.086563
4,11:00,43.269904,43.819673,0.184333,27.241791,50.306851
5,12:00,48.963407,49.554130,0.133251,24.328236,51.604390
6,13:00,70.766818,71.976855,0.019517,16.339508,53.030798
7,14:00,86.268547,87.945547,0.026738,15.989767,51.973171
8,15:00,82.709463,84.168204,0.164756,23.040625,51.527826
9,16:00,97.113167,99.320370,0.112270,20.215995,49.768556


In [135]:
datos31[["hour","PM2","PM10","CO","NO","SO"]].sort_values(by="hour")

,hour,PM2,PM10,CO,NO,SO
15,8,43.5366,63.3896,0,42.224,4.1055
14,9,45.8280,64.1920,0,43.526,4.2075
13,10,42.7125,66.0690,0,47.425,4.6092
12,11,44.7225,68.1150,0,52.850,4.7070
11,12,47.7375,70.5000,0,52.899,4.3470
10,13,50.2500,71.7000,0,59.270,5.0300
9,14,52.0400,73.4100,0,59.890,5.2300
8,15,57.9500,80.2400,0,60.320,4.7900
7,16,66.0200,95.2600,0,62.180,4.7600
6,17,73.2300,109.0200,0,67.750,5.1700


In [136]:
acumulado = df_merge.query("fecha.dt.day==31").copy()

In [139]:
acumulado = acumulado.query("name == 'MATERIAL PARTICULADO 10'")

In [153]:
acumulado.loc[(acumulado.fecha.dt.hour>7 & acumulado.fecha.dt.hour <=8)].value.mean()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [154]:
 acumulado.loc[(acumulado.fecha.dt.hour > 7) & (acumulado.fecha.dt.hour <= 8), "value"].mean()

97.51769811320754

In [155]:
pivot_df

name,fecha,hora,PM2.5,PM10,CO,NO2,SO2
0,2024-08-31 07:00:00,07:00,94.848150,110.330450,0.000000,6.473047,53.355828
1,2024-08-31 08:00:00,08:00,85.546528,97.517698,1.040386,35.222671,49.160716
2,2024-08-31 09:00:00,09:00,66.480642,71.134189,1.896588,49.837261,50.277353
3,2024-08-31 10:00:00,10:00,56.346660,58.750849,0.375608,33.769560,49.086563
4,2024-08-31 11:00:00,11:00,43.269904,43.819673,0.184333,27.241791,50.306851
5,2024-08-31 12:00:00,12:00,48.963407,49.554130,0.133251,24.328236,51.604390
6,2024-08-31 13:00:00,13:00,70.766818,71.976855,0.019517,16.339508,53.030798
7,2024-08-31 14:00:00,14:00,86.268547,87.945547,0.026738,15.989767,51.973171
8,2024-08-31 15:00:00,15:00,82.709463,84.168204,0.164756,23.040625,51.527826
9,2024-08-31 16:00:00,16:00,97.113167,99.320370,0.112270,20.215995,49.768556


In [157]:
datos31.sort_values(by="hour")

,hora,PM2,PM10,SO,NO,CO,hour
15,08:00:00,43.5366,63.3896,4.1055,42.224,0,8
14,09:00:00,45.8280,64.1920,4.2075,43.526,0,9
13,10:00:00,42.7125,66.0690,4.6092,47.425,0,10
12,11:00:00,44.7225,68.1150,4.7070,52.850,0,11
11,12:00:00,47.7375,70.5000,4.3470,52.899,0,12
10,13:00:00,50.2500,71.7000,5.0300,59.270,0,13
9,14:00:00,52.0400,73.4100,5.2300,59.890,0,14
8,15:00:00,57.9500,80.2400,4.7900,60.320,0,15
7,16:00:00,66.0200,95.2600,4.7600,62.180,0,16
6,17:00:00,73.2300,109.0200,5.1700,67.750,0,17
